# **Dataset**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics

Cargar los Datos.

In [2]:
movies_df = pd.read_csv("movies.csv").drop(['genres'],axis =1)
df = pd.read_csv("movies_rating.csv").drop(['Unnamed: 0'],axis =1)
df_rating=df

Dividir los datos en conjuntos de datos de entrenamiento y prueba.

In [4]:
min_rating = min(df["rating"])
max_rating = max(df["rating"])
X = df_rating[["userId", "movieId"]]
y = df[['rating']].apply(lambda x: (x - min_rating) / (max_rating - min_rating))
y = y.astype('int64')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Diccionario CLAVE - VALOR 

In [5]:
user_mapper = dict(df_rating.movieId) 
movies_list = list(user_mapper)

Funciones para retornar los valores de fila o columna de cada película.

In [6]:
def obtener_fila(val):
    for key, value in user_mapper.items():
         if val == value:
             return key

In [7]:
def obtener_valor(val):
    for key, value in user_mapper.items():
         if val == key:
             return value

# **KNN Model**
Uno de los métodos más populares de filtrado colaborativo es el algoritmo k-Nearest Neighbors (kNN). kNN es un algoritmo de clasificación donde k es un parámetro que especifica el número de vecinos utilizados. Para usar kNN, el sistema debe tener una medida similitud para distinguir entre los usuarios que están cerca y los que están lejos. 

In [29]:
# Librerías necesarias para el modelo
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error

In [84]:
KNN = KNeighborsClassifier(metric='cosine', algorithm='brute', n_neighbors=5)
KNN.fit(X_train, y_train.values.ravel())    
y_predict = KNN.predict(X_test)

def recomendacion_modeloKNN(id, modelo, datos, k):
  fila=obtener_fila(id)
  movies_list = list(user_mapper)
  print('Pelicula Seleccionada: ',fila)  
  print('-----------------------------------------------------------------')
  distancia,indice=modelo.kneighbors(X_test.iloc[[fila]], n_neighbors = k)   
  print(indice)     

Modelo de recomendación KNN

In [85]:
recomendacion_modeloKNN(1, KNN, X_train,5)

Pelicula Seleccionada:  0
-----------------------------------------------------------------
[[ 8843 37104 60642   487 30683]]


Métricas de evaluación del modelo KNN

In [86]:
print("KNN Accuracy:",metrics.accuracy_score(y_test, y_predict))
print("Root Mean Square error KNN",sqrt(mean_squared_error(y_test,y_predict)))

KNN Accuracy: 0.8568972926514826
Root Mean Square error KNN 0.3782891848156875


Exactitud para cada K del modelo

In [87]:
scores = cross_val_score(KNN, X, y, cv=5, scoring='accuracy')
print(scores)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

[0.76190004 0.85496108 0.85193633 0.85540735 0.85144047]


Promedio de la exactitud del modelo

In [90]:
scores.mean()

0.8351290534036826

# **SVD Model**
La Descomposición en valores singulares (SVD) es un algoritmo conveniente para trabajar con matrices. La Descomposición en valores singulares se utiliza en la resolución de varios problemas, el método de aproximación mínimos cuadrados, resolución de sistemas de ecuaciones, entre otras.

In [8]:
# Librerías necesarias para el modelo
!pip install surprise
from collections import defaultdict
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, accuracy

In [10]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_rating, reader)
trainset, testset = train_test_split(data, test_size=0.30)

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

def recomendacion_modeloSVD(predictions, n):
    # Asignar las predicciones a cada usuario.
    top_n = defaultdict(list)    
    print('registros',top_n.items())
    for uid, iid, true_r, est, _ in predictions:
        top_n[0].append((iid, est))

    # Ordenar las predicciones para cada usuario y recupere las k más altas.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

Modelo de recomendación SVD

In [ ]:
top_n = recomendacion_modeloSVD(predictions, n=10)

# Películas recomendadas para el usuario
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

Métrica de evaluación del modelo KNN

In [11]:
print("Root Mean Square Error SVD",accuracy.rmse(predictions))

RMSE: 0.8766
Root Mean Square Error SVD 0.8766428735313772


# **Matrix Factorization Model**
La Factorización Matricial determina los valores de la calificación pronosticada al separar los componentes matrices del usuario y encontrar el valor del parámetro más cercano posible a la matriz original. El cual será el valor que tenga calificaciones de películas que los usuarios nunca hayan calificado antes y sugerirá películas con calificaciones altas.

In [15]:
# Librerías necesarias para el modelo
!pip install thunder-factorization
!pip install git+https://github.com/maciejkula/spotlight.git lightfm
from spotlight.interactions import Interactions
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import rmse_score

  Cloning https://github.com/maciejkula/spotlight.git to /tmp/pip-req-build-ep0e_mbs
  Running command git clone -q https://github.com/maciejkula/spotlight.git /tmp/pip-req-build-ep0e_mbs


In [16]:
dataset = Interactions(user_ids=df_rating['userId'].values,   
                       item_ids=df_rating['movieId'].values,                    
                       ratings=df_rating['rating'].values)
train, test = random_train_test_split(dataset)

modelex = ExplicitFactorizationModel(n_iter=10)
modelex.fit(train, verbose=True)

Epoch 0: loss 4.349146908268966
Epoch 1: loss 0.809280066641488
Epoch 2: loss 0.5015158361835762
Epoch 3: loss 0.3612660800567194
Epoch 4: loss 0.29030551649372405
Epoch 5: loss 0.25598594820073034
Epoch 6: loss 0.23492095927890086
Epoch 7: loss 0.22508255981138095
Epoch 8: loss 0.21382118655862387
Epoch 9: loss 0.20470441905520484


In [17]:
def recomendacion_modeloMF(user_id, modelex, n_movies=5):    
      
    pred = modelex.predict(user_ids=user_id)
    indices = np.argpartition(pred, -n_movies)[-n_movies:]
    best_movie_ids = indices[np.argsort(pred[indices])]
    unico = []
    
    for i in best_movie_ids:
        print(i)     

Modelo de recomendación Factorización Matricial

In [18]:
recomendacion_modeloMF(5, modelex, n_movies=5)

596
55363
1223
2530
2387


Métrica de evaluación del modelo Factorización Matricial

In [23]:
print("Root Mean Square Error Factorización Matricial:",rmse_score(modelex, test))

Root Mean Square Error Factorización Matricial: 1.0682667993294057
